In [3]:
# imports
import os
import pickle
import torch
import importlib
import constants
import utils
import rlenv.env_consts as env_consts
import rlenv.model_names as model_names
import pandas as pd
from simulator.nets import FeedForward
importlib.reload(constants)
importlib.reload(env_consts)
importlib.reload(utils)
importlib.reload(model_names)


<module 'rlenv.model_names' from 'A:\\Dropbox\\eBay\\repo\\rlenv\\model_names.py'>

### Attempting to load model

In [2]:
path = 'models'
os.chdir('A:/dropbox/ebay/') # BARRY'S Working directory

In [4]:
# loads a model given its name
def load_model(model_type, model_name):
    dir_path = '{}/{}/{}/'.format(path, model_type, model_name)
    info_path = '{}featnames.pkl'.format(dir_path)
    model_path = '{}1.pt'.format(dir_path )
    info = pickle.load(open(info_path, 'rb'))
    model = torch.load(model_path)
    return info, model

def load_params(model_type, model_name):
    dir_path = '{}/{}/{}/'.format(path, model_type, model_name)
    params_path = '{}params.csv'.format(dir_path)
    sizes_path = '{}sizes.pkl'.format(dir_path)
    params = pd.read_csv(params_path, index_col='id')
    sizes = utils.unpickle(sizes_path)
    return params, sizes

def load_featnames(model_type, model_name):
    dir_path = '{}/{}/{}/'.format(path, model_type, model_name)
    featnames_path = '{}featnames.pkl'.format(dir_path)
    featnames_dict = utils.unpickle(featnames_path)
    return featnames_dict

def load_sizes(model_type, model_name):
    dir_path = '{}/{}/{}/'.format(path, model_type, model_name)
    featnames_path = '{}sizes.pkl'.format(dir_path)
    featnames_dict = utils.unpickle(featnames_path)
    return featnames_dict

# gets types of everything in dict
def dict_types(input_dict):
    for key, value in input_dict.items():
        print('{}: {}'.format(key, type(value)))
        
        
def print_model_fixed(model_type, model_name):
    info, model = load_model(model_type, model_name)
    for info_element in info:
        for key, val in info_element.items():
            print('feature type: {}'.format(key))
            try:
                for col in val:
                    print(col)
            except:
                print(val)
    return set([el for el in info[0]['x_fixed']])
    

In [ ]:
# arrival bin
ab_info, ab_model = load_model('arrival', 'bin')

In [ ]:
ab_info

In [ ]:
ab_params, ab_sizes = load_params('arrival', 'bin')

In [ ]:
ab_params

In [ ]:
params = ab_params.loc[1].to_dict()

In [ ]:
net = FeedForward(params, ab_sizes, toRNN=False)

In [ ]:
params

In [ ]:
ab_sizes

In [ ]:
state_dict = torch.load('models/arrival/bin/1.pt')

In [ ]:
for el in state_dict:
    print(el)

In [ ]:
for el in net.state_dict():
    print(el)

In [ ]:
net.load_state_dict(torch.load('models/arrival/bin/2.pt'))

In [ ]:
def outer(n):
    for i in range(n):
        yield from inner(n, [i])
        
def inner(n, board):
    if len(board) == n:
        yield board
    else:
        for i in range(n):
            if i not in board:
                yield from inner(n, board + [i])
            

In [ ]:
k = outer(4)

In [ ]:
for i, el in enumerate(k):
    print(el)

In [ ]:
print(type(ab_info[0]))
print(type(ab_info[1]))

In [ ]:
print(ab_info[0])

In [ ]:
for key, val in ab_info[0].items():
    print('key:{}'.format(key))
    for col in val:
        print(col)
        
for key, val in ab_info[1].items():
    print('key:{}'.format(key))
    print(val)

In [ ]:
print_model_fixed('arrival', 'hist')
print_model_fixed('arrival', 'sec')
print_model_fixed('arrival', 'bin')

In [ ]:
ab.load_state_dict(torch.load('{}/{}_1.pt'.format(path, 'arrival_bin')))

In [ ]:
# bin exploration
sm_info, sm_model = load_model('slr', 'msg')

In [ ]:
print(sm_info)

In [ ]:
days_featnames, days_model = load_model('arrival', 'days')

In [ ]:
days_featnames

In [ ]:
for feat in days_featnames['x_fixed']:
    print(feat)

In [ ]:
delay_featnames, delay_model = load_model('slr', 'delay')

In [ ]:
print(len(delay_featnames['x_fixed']))
for feat in delay_featnames['x_fixed']:
    print(feat)

### Exploring RL Inputs

In [ ]:
data_path = 'data/partitions/train_rl'
x_path = '{}/x_lstg_cols.pkl'.format(data_path)

In [ ]:
x = utils.unpickle(x_path)

In [ ]:
print(type(x))
print(len(x))

In [ ]:
x_lstg = [el for el in x]
ab_info_inputs = [el for el in ab_info[0]['x_fixed']]
x_lstg = set(x_lstg)
ab_info_inputs = set(ab_info_inputs)

In [ ]:
# elements in arrival bin model that aren't in the x_lstg object
# significant that it includes dow0,...and focal_dow0
print(ab_info_inputs.difference(x_lstg))

In [ ]:
days = [el for el in ab_info_inputs if 'days' in el]
days

In [ ]:
# elements in x_lstg object that aren't in arrival bin model
print(x_lstg.difference(ab_info_inputs))

In [ ]:
for el in x:
    print(el)

In [ ]:
# prev_turn feature exploration
meta_featnames = dict()
for model in model_names.MODELS:
    if model in model_names.ARRIVAL:
        meta_featnames[model] = load_featnames('arrival', model)
    else:
        if model_names.SLR_PREFIX in model:
            model_type = model_names.SLR_PREFIX
        else:
            model_type = model_names.BYR_PREFIX
        meta_featnames[model] = load_featnames(model_type, model.replace('{}_'.format(model_type), ''))


In [ ]:
len(meta_featnames)

In [ ]:
other_feats = set()
last_feats = set()
for model_name, model in meta_featnames.items():
    fixed = model['x_fixed']
    for feat in fixed:
        if '_other' in feat:
            other_feats.add(feat)
        elif '_last' in feat:
            last_feats.add(feat)
    if 'x_time' in model:
        time = model['x_time']
        for feat in time:
            if '_other' in feat:
                other_feats.add(feat)
            elif '_last' in feat:
                last_feats.add(feat)
        

In [ ]:
print(len(last_feats))
last_feats

In [ ]:
print(len(other_feats))
other_feats

In [ ]:
other_no_diffs = set([feat for feat in other_feats if 'diff' not in feat])
print(len(other_no_diffs))
other_no_diffs

In [ ]:
other_no_diffs_prefix = set([feat.replace('_other', '') for feat in other_no_diffs])
last_prefix = set([feat.replace('_last', '') for feat in last_feats])

In [ ]:
print(other_no_diffs_prefix.difference(last_prefix))
print(last_prefix.difference(other_no_diffs_prefix))

In [ ]:
diffs = set()
for model_name, model in meta_featnames.items():
    fixed = model['x_fixed']
    for feat in fixed:
        if 'diff' in feat:
            diffs.add(feat)
    if 'x_time' in model:
        time = model['x_time']
        for feat in time:
            if 'diff' in feat:
                diffs.add(feat)

In [ ]:
diffs

In [ ]:
 # find diff feats
for model_name, model in meta_featnames.items():
    print('diffs in fixed for {}'.format(model_name))
    print('')
    fixed = model['x_fixed']
    for feat in fixed:
        if 'diff' in feat:
            print(feat)
            diffs.add(feat)
    if 'x_time' in model:
        print('diffs in time for {}'.format(model_name))
        print('')
        time = model['x_time']
        for feat in time:
            if 'diff' in feat:
                print(feat)
                diffs.add(feat)
    print('')
    print('')

In [ ]:
rej = load_featnames('slr', 'reject')

In [ ]:
for feat in rej['x_time']:
    print(feat)

In [ ]:
# find time feats for last
time_feats = ['best', 'best_open', 'offers', 'offers_open']
byr_feats = ['byr_{}'.format(feat) for feat in time_feats]
slr_feats = ['slr_{}'.format(feat) for feat in time_feats]
time_feats = byr_feats + slr_feats
for model_name, model in meta_featnames.items():
    if 'x_time' in model:
        print('last time feats for {}'.format(model_name))
        print('')
        time = model['x_time']
        for feat in time:
            if '_last' in feat:
                generalized = feat.replace('_last', '')
                if generalized in time_feats:
                    print(generalized)
    print('')
    print('')

In [ ]:
# search for period feats
for model_name, model in meta_featnames.items():
    if 'x_time' in model:
        print('fixed period feats for {}'.format(model_name))
        for feat in model['x_fixed']:
            if 'period' in feat:
                print(feat)
        print('')
        print('period feats for {}'.format(model_name))
        if 'x_time' in model:
            time = model['x_time']
            for feat in time:
                if 'period' in feat:
                    print(feat)
    print('')
    print('')

In [ ]:
fixed_sets = dict()
for model_name, model in meta_featnames.items():
    if ('slr' in model_name or 'byr' in model_name) and 'delay' not in model_name:
        curr_set = set()
        for feat in model['x_fixed']:
            curr_set.add(feat)
        fixed_sets[model_name] = curr_set

In [ ]:
for i in fixed_sets.values():
    print(len(i))

In [ ]:
intersection = set()
start = True
for i in fixed_sets.values():
    if start:
        intersection = i
        start = False
    else:
        intersection = intersection.intersection(i)
print(len(intersection))

In [ ]:
for feat in fixed_sets['slr_accept']:
    if feat in ['days', 'dow0', 'dow1', 'dow2', 'dow3', 'dow4', 'dow5', 'holiday', 'minutes']:
        print(feat)

In [ ]:
fixed_sets['slr_accept']

In [ ]:
time_sets = dict()
for model_name, model in meta_featnames.items():
    if ('slr' in model_name or 'byr' in model_name):
        curr_set = set()
        for feat in model['x_time']:
            curr_set.add(feat)
        time_sets[model_name] = curr_set

In [ ]:
for i in time_sets.values():
    print(len(i))

In [ ]:
time = time_sets['slr_delay']
time.remove('period')
print(len(time))
for set_vals in time_sets.values():
    time = time.intersection(set_vals)
print(len(time))

In [ ]:
for key in time_sets.keys():
    print(key)

In [ ]:
slr_accept = time_sets['slr_accept']
byr_accept = time_sets['byr_accept']
print(len(slr_accept.intersection(byr_accept)))

In [ ]:
slr_accept.difference(byr_accept)

In [ ]:
byr_accept.difference(slr_accept)

In [ ]:
 for model_name, feats in time_sets.items():
    if 'slr_' in model_name and 'delay' not in model_name:
        acc_minus = slr_accept.difference(feats)
        curr_minus = feats.difference(slr_accept)
        print('In accept, not {}: {}'.format(model_name, acc_minus))
        print('')
        print('In {}, not accept: {}'.format(model_name, curr_minus))
        print('')

In [ ]:
for feat in meta_featnames['slr_accept']['x_time']:
    print(feat)

In [ ]:
# time feats
time_feats = set()
for model_name, model in meta_featnames.items():
    if 'x_time' in model:
        time = model['x_time']
        for feat in time:
            if '_other' not in feat and '_last' not in feat and feat in env_consts.TIME_FEATS:
                time_feats.add(feat)
                
for model_name, model in meta_featnames.items():
    if 'x_time' in model:
        time = model['x_time']
        curr_feats = set()
        for feat in time:
            if '_other' not in feat and '_last' not in feat and feat in env_consts.TIME_FEATS:
                curr_feats.add(feat)
        print(curr_feats.difference(time_feats))
        print(time_feats.difference(curr_feats))

In [ ]:
time_feats

In [ ]:
from utils import unpickle

In [ ]:
sizes = unpickle('models/slr/accept/sizes.pkl')

In [ ]:
sizes

In [ ]:
meta_featnames['slr_delay']['x_time']

In [ ]:
delay_featnames = {
    'slr_delay': meta_featnames['slr_delay'],
    'byr_delay': meta_featnames['byr_delay']
}
print(delay_featnames)

In [ ]:
time_feats = {
    'slr': set(),
    'byr': set()
}
fixed_feats = {
    'slr': set(),
    'byr': set()
}
for model_name, model in delay_featnames.items():
    for feat in model['x_time']:
        if 'byr' in model_name:
            time_feats['byr'].add(feat)
        else:
            time_feats['slr'].add(feat)
    for feat in model['x_fixed']:
        if 'byr' in model_name:
            fixed_feats['byr'].add(feat)
        else:
            fixed_feats['slr'].add(feat)
    

In [ ]:
time_feats['slr']

In [ ]:
time_feats['byr']

In [ ]:
print(time_feats['byr'].difference(time_feats['slr']))
time_feats['slr'].difference(time_feats['byr'])

In [ ]:
fixed_feats['byr'].difference(fixed_feats['slr'])

for feat in fixed_feats['byr']:
    if len(feat) == 2:
        print(feat)

In [ ]:
for feat in fixed_feats['slr']:
    if 'other' in feat:
        print(feat)

In [ ]:
other_feats = [feat for feat in fixed_feats['slr'] if 'other' in feat]
len(other_feats)

In [ ]:
for feat in fixed_feats['byr']:
    if feat in outcomes:
        print(feat)

In [ ]:
time_other = ['{}_other'.format(feat) for feat in env_consts.TIME_FEATS]
clock_other = ['{}_other'.format(feat) for feat in env_consts.OFFER_CLOCK_FEATS]
outcomes_other = ['{}_other'.format(feat) for feat in env_consts.OUTCOMES]
for feat in fixed_feats['slr']:
    if '_other' in feat and feat not in time_other and feat not in clock_other and feat not in outcomes_other:
        print(feat)
print('clock: {}'.format(len(clock_other)))
print('time: {}'.format(len(time_other)))
print('outcomes: {}'.format(len(outcomes_other)))

In [ ]:
for i in meta_featnames:
    print(i)

In [ ]:
feats = load_featnames('slr', 'accept')
time_feats = feats['x_time']
time_feats = pd.DataFrame(index=time_feats, data={'to': np.arange(len(time_feats))})

In [ ]:
import numpy as np

In [ ]:
time_feats.index[time_feats['to'].isin(list(range(50, 58)))]

In [ ]:
outcomes = ['{}_other'.format(feat) for feat in env_consts.OUTCOMES]
outcomes = set(outcomes)
contained = set()
for feat in feats['x_time']:
    if feat in outcomes:
        contained.add(feat)

In [ ]:
outcomes.difference(contained)

In [ ]:
a = torch.tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 14, 12, 16, 11, 15, 13, 17, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 18, 22, 20, 24, 19, 23, 21, 25, 50, 54,
        52, 56, 51, 55, 53, 57, 32, 26, 27, 28, 29, 30, 31, 67, 68,  0, 65, 60,
        66, 58, 59, 61, 62, 63, 64]).numpy()
diff = np.setdiff1d(np.arange(len(time_feats)), a)

In [ ]:
time_feats[time_feats['to'].isin(diff)]

In [140]:
slr_delay = load_featnames('slr', 'delay')
byr_delay = load_featnames('byr', 'delay')
rm_slr = ['auto_other', 'exp_other', 'reject_other']
rm_byr = ['auto_last', 'exp_last', 'reject_last']

In [141]:
new_slr_delay = slr_delay['x_fixed'].drop(rm_slr)

KeyError: "['auto_other' 'exp_other' 'reject_other'] not found in axis"

In [138]:
print(len(slr_delay['x_fixed']))
print(len(new_slr_delay))

179
179


In [134]:
new_byr_delay = byr_delay['x_fixed'].drop(rm_byr)

In [137]:
print(len(byr_delay['x_fixed']))
print(len(new_byr_delay))

180
180


In [136]:
slr_delay['x_fixed'] = new_slr_delay
byr_delay['x_fixed'] = new_byr_delay

In [139]:
pickle.dump(slr_delay, open('models/slr/delay/featnames.pkl', 'wb'))
pickle.dump(byr_delay, open('models/byr/delay/featnames.pkl', 'wb'))

In [142]:
slr_delay['x_time']

Index(['period', 'holiday', 'dow0', 'dow1', 'dow2', 'dow3', 'dow4', 'dow5',
       'minutes', 'slr_offers', 'slr_best', 'slr_offers_open', 'slr_best_open',
       'byr_offers', 'byr_best', 'byr_offers_open', 'byr_best_open'],
      dtype='object')

In [143]:
byr_delay['x_time']

Index(['period', 'holiday', 'dow0', 'dow1', 'dow2', 'dow3', 'dow4', 'dow5',
       'minutes', 'slr_offers', 'slr_best', 'slr_offers_open', 'slr_best_open',
       'byr_offers', 'byr_best', 'byr_offers_open', 'byr_best_open'],
      dtype='object')

In [146]:
slr_sizes = load_sizes('slr', 'delay')
byr_sizes = load_sizes('byr', 'delay')

In [148]:
slr_sizes['fixed'] -= 3
byr_sizes['fixed'] -= 3

In [149]:
print(slr_sizes)
print(byr_sizes)

{'fixed': 179, 'out': 1, 'steps': 193, 'time': 17}
{'fixed': 180, 'out': 1, 'steps': 193, 'time': 17}


In [150]:
pickle.dump(slr_sizes, open('models/slr/delay/sizes.pkl', 'wb'))
pickle.dump(byr_sizes, open('models/byr/delay/sizes.pkl', 'wb'))